# What does sample, batch, epoch mean?

Below are common definitions that are necessary to know and understand to correctly utilize keras:  
* **Sample**: one element of a dataset.  
* **Batch**: a set of N samples. The samples in a batch are processed independently, in parallel. If training, a batch results in only update to the model.  
  - A batch generally approximates the distribution of the input data better than a single input. The larger the batch, the better the approximation; however, it is also true that the batch will take longer to process and will still result in only one update. For inference (evaluate/predict), it is recommended to pick a batch size that is as large as you can afford without going out of memory (since larger batches will usually result in faster evaluation/prediction).  
* **Epoch**: an arbitrary cutoff, generally defined as "one pass over the entire dataset", used to separate training into distinct phases, which is useful for logging and periodic evaluation.  
  - When using validation_data or validation_split with the fit method of keras models, evaluation will be run at the end of every epoch.  
  - Within Keras, there is the ability to add callbacks specifically designed to be run at the end of an **epoch**. Examples of these are learning rate changes and model checkpointing(saving).

# How can Save a Keras model?

**Saving/loading whole models (architeture + weights + optimizer state)**  
It is not recommended to use pickle or cPickle to save keras model.  
You can use *model.save(filepath)* to save a keras model into a single HDF5 file which will contain:  
* The architeture of the model, allowing to re-create the model.  
* The weights of the model.  
* The training configuration (loss, optimizer).  
* The state of the optimizer, allowing to resume training exatly where you left off.  

You can then use *keras.models.load_model(filepath)* to reinstantiate your model. Load_model will also take care of compiling the model using the saved training configuration (unless the model was never compiled in the first place).  
```python
from keras.models import load_model

# creates a HDF5 file
model.save('my_model.h5')
del model # deletes the existing model

# returns a compiled model
# identical to previous one
model = load_model('my_model.h5')
```

Please also see How can i install HDF5 or h5py to save my models in Keras? For instructions on how to install h5py

**Saving/loading only a model's architeture**  
If you only need to save the architecture of a model, and not its weights or its training configuration, you can do:  

```python
# save as JSON
json_string = model.to_json()

# save as YAML
yaml_string = model.to_yaml()
```

The generated JSON/YAML files are human-readable and can be manually edited if needed.  

You can then build a fresh model from this data:  

```python
# model reconstruction from JSON
from keras.models import model_from_json
model = model_from_json(json_string)

# model reconstruction from YAML
from keras.models import model_from_yaml
model = model_from_yaml(yaml_string)
```

**Saving/Loading only a model's weights**  
If you need to save the weights of a model, you can do so in HDF5 with the code below:  
```python
model.load_weights('my_model_weights.h5')
```

Assuming you have code for instantiating your model, you can then load the weights you saved into a model with the same architeture:  
```python
model.load_weights('my_model_weights.h5', by_name=True)
```

Example:  
```python
# Assming the original model looks like this
model = Sequential()
model.add(Dense(2, input_dim=3, name='dense_1'))
model.add(Dense(3, name='dense_2'))
...
model.save_weights(fname)

# new model
model = Sequential()
model.add(Dense(2, input_dim=3, name='dense_1')) # will be loaded
model.add(Dense(10, name='new_dense')) # will not be loaded

# load weights from first model, will only affect the first layer, dense_1
model.load_weights(fname, by_name=True)
```

**Handling custom layers (or other custom objects) in saved models**  
If the model you want to load includes custom layers or other custom classes or functions, you can pass them to the loading mechanism via the custom_objects argument:  

```python
from keras.models import load_model
# Assuming your model includes instance of an "AttentionLayer" class
model = load_model('my_model.h5', custom_objects={'AttentionLayer': AttentionLayer})
```

Alternatively, you can use a custom object scope:  
```python
from keras.utils import CustomObjectScope

with CustomObjectScope({'AttentionLayer': AttentionLayer}):
  model = load_model('my_model.h5')
```

Custom objects handling works the same way for load_model, model_from_json, model_from_yaml:  
```python
from keras.models import model_from_json
model = model_from_json(json_string, custom_objects={'AttentionLayer': AttentionLayer})
```

# Why is the training loss much higher than the testing loss?
A keras model has two modes: training and testing. Regularization mechanism, suchs as Dropout and L1/L2 weight regularization, are turned off at testing time.  

Besides, the training loss is the average of the losses over each batch of training data. Because your model is changing over time, the loss over the first batchesof an epoch is generally higher than over the last batches. On the other hand, the testing loss for an epoch is computed using the model as it is at the end of the epoch, resulting in a lower loss.  

[More information](https://www.pyimagesearch.com/2019/10/14/why-is-my-validation-loss-lower-than-my-training-loss/)

# How can I obtain the output of an intermediate layer?
One simple way is to create a new Model that will output the layers that you are interested in:  
```python
from keras.models import Model
model = .... # create the original model
layer_name = 'my_layer'
intermediate_layer_model = Model(inputs=model.input,
                                outputs=model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(data)                          
```

Alternatively, you can build a Keras function that will return the output of a certain layer given a certain input, for example:  
```python
from keras import backend as k
# with a Sequential model
get_3rd_layer_output = k.function([model.layers[0].input],
                                  [model.layers[3].output])
layer_output = get_3rd_layer_output([X])[0]                                  
```

Similarly, you could build a Theano and TensorFlow function directly.  
Note that if your model has a different behavior in training and testing phase (e.g. if it uses Dropout, BatchNormalization, etc), you will need to pass the learning phase flag to your function:  
```python
get_3rd_layer_output = k.function([model.layers[0].input, k.learning_phase()],
                                  [model.lauyers[3].output])
# output in test mode = 0
layer_output = get_3rd_layer_output([x, 0])[0]
# outpuyt in train mode = 1
layer_output = get_3rd_layer_output([x, 1])[0]                                  
```

# How can I use Keras with datasets that don't fit in memory?
You can do batch training using *model.train_on_batch(x, y)* and *model.test_on_batch(x, y)*. See the models documentation.  

Alternatively, you can write a generator that yields batches of training data and use the method *model.fit_generator(data_generator, steps_per_epoch, epochs)*.

# How can I interrupt training when the validation loss isn't decreasing anymore?
You can use an EarlyStopping callback:  
```python
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
model.fit(x, y, validation_split=0.2, callbacks=[early_stopping])
```

# How is the validation split computed?
If you set the validation_split argument in model.fit to e.g. 0.1, then the validation data used will be the last 10% of the data. If you set it to 0.25, it will be the last 25% of the data, etc. Note that the data isn't shuffled before extracting the validation split, so the validation is literally just the last x% of samples in the input you passed.  

The same validation set is used for all epochs (within a same call to fit).

# Is the data shuffled during training?
Yes, if the shuffle argument in model.fit is set to True (which is the default), the training data will be randomly shuffled at each epoch.  

Validation data is never shuffled

# How can I record the training / validation loss / accuracy at each epoch?
The model.fit method returns a History callback, which has a history attribute the lists of successive losses and other metrics.  
```python
hist = model.fit(x, y, validation_split=0.2)
print(hist.history)
```

# How can I "freeze" keras layers?
To freeze a layer means to exclude it from training, i.e. its weights will never be updated. This is useful in the context of fine-tuning a model, or using fixed embeddings for a text input.  

You can pass a trainable argument (boolean) to a layer constructor to set a layer to be non-trainable:  
```python
frozen_layer = Dense(32, trainable=False)
```

Additionally, you can set the trainable property of a layer to True or False after instantiation. FOr this to take effect, you will need yo call compile() on your model after modifying the trainable property.  
```python
x = Input(shape=(32,))
layer = Dense(32
layer.trainable = False
y = layer(x)

frozen_model = Model(x, y)
# in the model below, the weights of layer will not be updated during training
frozen_model.compile(optimizer='rmsprop', loss='mse')
layer.trainable = True 
trainable_model = Model(x, y)

# with this model the weights of the layer will be updated during training 
# which will also affect the above model since it uses the same layer instance
trainable_model.compile(optimizer='rmsprop', loss='mse')

frozen_model.fit(data, labels) # this does NOT update update the weights of layer
trainable_model.fit(data, labels) # this updates the weights of layer

# How can I remove a layer from a Sequential model?
You can remove the last added layer in a Sequential model by calling .pop().  
```python
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=784))
model.add(Dense(32, activation='relu'))

print(len(model.layers)) # 2
model.pop()
print(len(model.layers)) # 1
```

# How can I use pre-train models in keras?
Code and pre-trained weights are available for the following image classification models:  
* Xception  
* VGG16  
* VGG19  
* ResNet  
* ResNet v2  
* ResNext  
* Inception-ResNet v2  
* MobileNet v1  
* MobileNet v2  
* DenseNet  
* NASNet  

They can be imported from the module keras.applications:  
```python
from keras.applications.xception import Xception
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet import ResNet50
from keras.applications.resnet import ResNet101
from keras.applications.resnet import ResNet152
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.resnet_v2 import ResNet101V2
from keras.applications.resnet_V2 import ResNet152V2
from keras.applications.resnext import ResNext50
from keras.applications.resnext import ResNext101
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.densenet import DenseNet121
from keras.applications.densenet import DenseNet169
from keras.applications.densenet import DenseNet201
from keras.applications.nasnet import NASNetLarge
from keras.applications.nasnet import NASNetMobile

model = VGG16(weights='imagenet', include_top=True)
```

For a few simple usage examples, see the documentation for the Applications module.  

# How can I use HDF5 inputs with Keras?
Youy can use the HDF5Matrix class from keras.utils. See the HDF5Matrix documentation for details.  

You can also directly use a HDF5 dataset:  
```python
import h5py
with h5py.File('file.hdf5', 'r') as f:
  x_data = f['x_data'])
  model.predict(x_data)
```